In [1]:
using ADCME
using Plots
using PyCall
#### Read data
py"""
import numpy as np
"""

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\Michael\.julia\packages\PyCall\BcTLp\src\pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'ImportError'>
ImportError('\n\nIMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!\n\nImporting the numpy C-extensions failed. This error can happen for\nmany reasons, often due to issues with your setup or how NumPy was\ninstalled.\n\nWe have compiled some common reasons and troubleshooting tips at:\n\n    https://numpy.org/devdocs/user/troubleshooting-importerror.html\n\nPlease note and check the following:\n\n  * The Python version is: Python3.7 from "C:\\Users\\Michael\\AppData\\Local\\Programs\\Julia 1.5.2\\bin\\julia.exe"\n  * The NumPy version is: "1.19.1"\n\nand make sure that they are the versions you expect.\nPlease carefully study the documentation linked above for further help.\n\nOriginal error was: DLL load failed: The specified module could not be found.\n')


In [5]:
function f_hi(a, T, R)
        v1 = (a[:,2] + a[:,3] * T + a[:,4] * T^2 + a[:,5] * T^3 + a[:,6] * T^4) * R
        v2 = (a[:,2] + a[:,3] * T / 2 + a[:,4] * T^2 /3 + a[:,5] * T^3 /4 + a[:,6] * T^4 /5 + a[:,7]/T) * R * T
        v3 = (a[:,2] * log(T) + a[:,3] * T + a[:,4] * T^2 /2 + a[:,5] * T^3 /3 + a[:,6] * T^4 / 4 + a[:,8]) * R
    return [v1 v2 v3]
end
 
function f_lo(a, T, R)
    v1 = (a[:,9] + a[:,10] * T + a[:,11] * T^2 + a[:,12] * T^3 + a[:,13] * T^4) * R
    v2 = (a[:,9] + a[:,10] * T / 2 + a[:,11] * T^2 /3 + a[:,12] * T^3 /4 + a[:,13] * T^4 /5 + a[:,14]/T) * R * T
    v3 = (a[:,9] * log(T) + a[:,10] * T + a[:,11] * T^2 /2 + a[:,12] * T^3 /3 + a[:,13] * T^4 / 4 + a[:,15]) * R
    return [v1 v2 v3]
end 

function compute_falloff(T, pr, a)
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(fcent) / log(10)
    n = 0.75 - 1.27 * log(fcent) / log(10)
    f1 = (log(pr) / log(10) + c) / (n - 0.14 *(log(pr) / log(10) + c))
    return  10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2))
end

function compute_mout_dot(m, cp, cv, R, X, W, Y, T)
    cp_total = sum(cp ./ W .* Y)
    _area = 1
    _gamma_s = cp_total / cv
    _press = sum(X) * R * T
    _r_gas_specific = cp_total - cv
    _p_ratio = 101325 / (_press + 1.0)
    power_1 = 2.0 / _gamma_s
    power_2 = (_gamma_s + 1.0) / (_gamma_s)
    pressure_term = _p_ratio ^ power_1 - _p_ratio ^ power_2
    _sqrt_term = 2. * _gamma_s * _r_gas_specific * T / (_gamma_s - 1.0)
    _sqrt_term *= pressure_term
    _sqrt_term = (_sqrt_term)^(0.5)
    _mdot_unchoked = m * _area * _sqrt_term
    power = (_gamma_s + 1.0) / (_gamma_s - 1.0)
    _gamma_term = (2. / (_gamma_s + 1.0)) ^ power
    _sqrt = (_gamma_s * _r_gas_specific * T * _gamma_term)^(0.5)
    _mdot_choked = m * _area * _sqrt
    _p_crit_downstream = _press * (2. / (_gamma_s + 1.0)) ^ (_gamma_s / (_gamma_s - 1.0))
    ischoked = ADCME.constant(_p_crit_downstream) >= ADCME.constant(101325.0)
    ischoked = convert_to_tensor(ischoked, dtype=Float64)
    mout_dot = ischoked * _mdot_choked + (1 - ischoked) * _mdot_unchoked 
    return mout_dot
end

function compute_ω_dot(m, T, Y, ρ, X)
    Q = ADCME.constant(ones(M))
    islarge = T > ADCME.constant(NASA_coeffs[:,1])
    islarge = convert_to_tensor(islarge, dtype=Float64)
    fhi = f_hi(NASA_coeffs, T, R)
    flo = f_lo(NASA_coeffs, T, R)
    cp = islarge .* fhi[:,1] + (1 - islarge) .* flo[:,1]
    h = islarge .* fhi[:,2] + (1 - islarge) .* flo[:,2]
    s = islarge .* fhi[:,3] + (1 - islarge) .* flo[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    M_t = efficiency_t' * X[:,1]
    Kf_t = Af_t .* (T ^ β_t) .* exp(-E_t / (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) ^ sum(ν[:,tbd], dims=1)')[:,1] .* exp(ΔS[tbd] ./ R - ΔH[tbd] / (R * T)))
    Qtbd =  Kf_t .* (prod(tf.pow(X, order_t), dims=1)) .- Kr_t .* (prod(tf.pow(X, ν2[:,tbd]), dims=1) .* reversible[tbd])
    M_f = efficiency_f' * X[:,1]
    Kf_lo = Af_lo .* (T ^ β_lo) .* exp(-E_lo / (R * T)) .* M_f
    Kf_hi = Af_hi .* (T ^ β_hi) .* exp(-E_hi / (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ADCME.constant(ones(size(falofr)[1]))
    for (i,s) in enumerate(troefall)
        a = troefall_coeff[:,i]
        fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
        c = -0.4 - 0.67 * log(fcent) / log(10)
        n = 0.75 - 1.27 * log(fcent) / log(10)
        f1 = (log(Pr[s]) / log(10) + c) / (n - 0.14 *(log(Pr[s]) / log(10) + c))
        Fac = scatter_update(Fac, troefall[i], 10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2)))
    end
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) ^ sum(ν[:,falofr], dims=1)')[:,1] .* exp(ΔS[falofr] ./ R - ΔH[falofr] / (R * T)))
    Qfalofr = Kf_f .* (prod(tf.pow(X, order_f), dims=1)) .- Kr_f .* (prod(tf.pow(X, ν2[:,falofr]), dims=1) .* reversible[falofr])
    ####
    Kf = Af .* (T ^ β) .* exp(-E / (R * T))
    Kr = Kf ./ (((pa/(R * T)) ^ sum(ν[:,elmr], dims=1)')[:,1] .* exp(ΔS[elmr] ./ R - ΔH[elmr] / (R * T)))
    Qelmr = Kf .* (prod(tf.pow(X, order), dims=1)) .- Kr .* (prod(tf.pow(X, ν2[:,elmr]), dims=1) .* reversible[elmr])
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h / W - R ./ W * T   # Internal energy for species
    p = sum(X) * R * T # pressure
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    return ω_dot, cp, cv, u, p
end

compute_ω_dot (generic function with 1 method)

In [6]:
V = 1  # Volume
m_ = (py"np.load"(".npy files/density.npy"))[1]  # Total mass
tbd = py"np.load"(".npy files/tbd.npy") .+ 1
falofr = py"np.load"(".npy files/falofr.npy") .+ 1
elmr = py"np.load"(".npy files/elmr.npy") .+ 1
NASA_coeffs = py"np.load"(".npy files/NASA_coeffs.npy")
W = py"np.load"(".npy files/molecular_weights.npy") # Molar weight
ν1 = py"np.load"(".npy files/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"(".npy files/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
reversible = py"np.load"(".npy files/reversible.npy")
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν1_order = zeros(N,M)
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.4621 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
Tt_cant = py"np.load"(".npy files/temperature.npy")
T = Tt_cant[1] # Temperature
Y = py"np.load"(".npy files/ini_mass_fraction.npy") # Mass fractions
#Y .+= 0.00001
#Y ./ sum(Y)
### Quantities that depend on the Unknowns
tbd = py"np.load"(".npy files/tbd.npy") .+ 1
tbd = Int.(tbd)
falofr = py"np.load"(".npy files/falofr.npy") .+ 1
falofr = Int.(falofr)
elmr = py"np.load"(".npy files/elmr.npy") .+ 1
elmr = Int.(elmr)

order = py"np.load"(".npy files/reaction_orders.npy")
Af = py"np.load"(".npy files/pre_exponential_factor.npy") # preexponential constant Afj
β = py"np.load"(".npy files/temperature_exponent.npy") # Temperature exponent
E = py"np.load"(".npy files/activation_energy.npy") # Activation energy for the reactions in kJ

order_t = py"np.load"(".npy files/reaction_orders_t.npy")
efficiency_t = py"np.load"(".npy files/efficiency_t.npy")
Af_t = py"np.load"(".npy files/pre_exponential_factor_t.npy") # preexponential constant Afj
β_t = py"np.load"(".npy files/temperature_exponent_t.npy") # Temperature exponent
E_t = py"np.load"(".npy files/activation_energy_t.npy") # Activation energy for the reactions in kJ

order_f = py"np.load"(".npy files/reaction_orders_f.npy")
troefall = py"np.load"(".npy files/troefall.npy") .+ 1
troefall_coeff = py"np.load"(".npy files/troefall_coeff.npy")
efficiency_f = py"np.load"(".npy files/efficiency_f.npy")
Af_hi = py"np.load"(".npy files/pre_exponential_factor_hi.npy") # preexponential constant Afj
β_hi = py"np.load"(".npy files/temperature_exponent_hi.npy") # Temperature exponent
E_hi = py"np.load"(".npy files/activation_energy_hi.npy") 

Af_lo = py"np.load"(".npy files/pre_exponential_factor_lo.npy") # preexponential constant Afj
β_lo = py"np.load"(".npy files/temperature_exponent_lo.npy") # Temperature exponent
E_lo = py"np.load"(".npy files/activation_energy_lo.npy") 

ν1_order[:,elmr] = order
ν1_order[:,tbd] = order_t
ν1_order[:,falofr] = order_f;

In [7]:
function q(t)
    t0 = 1e-5
    r = 1e-6
    c = 5e11
    return c * exp(-(t-t0)^2 / r^2)
end
function f(mTY, Qdot, t)
    m = mTY[1]
    T = mTY[2]
    Y = mTY[3:end]
    ρ = m / V # density
    X = tf.reshape(ρ * tf.divide(Y, W), (N,1)) # Concentration
    ω_dot, cp, cv, u, p = compute_ω_dot(m, T, Y, ρ, X)
    ###### mout_dot computation
    mout_dot = compute_mout_dot(m, cp, cv, R, X, W, Y, T)
    min_dot = 0
    m_dot = min_dot - mout_dot
    ###### Species Conservation
    mgen_dot = V * ω_dot
    Y_dot = (1 / m) * ((min_dot * (Yin - Y)) + mgen_dot) 
    ###### EnergyConservation
    #Qdot = abs(fc(t * 1e5, [20,20,1], theta[2]) + 1) * 10 ^ theta[1] 
    Qdot = 5e11 * exp(-(t - 1e-5) ^ 2 / (1e-6 * 1e-6))
    T_dot = 1 / (m * cv) * (Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return tf.concat([tf.reshape(m_dot, (1,)), tf.reshape(T_dot, (1,)), Y_dot], 0)
end

f (generic function with 1 method)

In [8]:
timestep = py"np.load"(".npy files/timestep.npy")
Tt_cant = py"np.load"(".npy files/temperature.npy")
T_ref = zeros(size(timestep)[1])
T_ref = Tt_cant
Y_ref = Yt_cant
Yt_cant = py"np.load"(".npy files/mass_fraction.npy")
obs = [T_ref Y_ref'];

LoadError: UndefVarError: Yt_cant not defined

In [9]:
num_s = 1
t = constant(timestep)
#θ = Variable(fc_init([1,20,20,1]))
θ = Variable(py"np.load"(".npy files/theta_.npy"))
#C = Variable(3.0)
C = Variable(py"np.load"(".npy files/c_.npy"))
Qdtb = abs(fc(t * 1e5, [20,20,1], θ))
Qdot = 1.0 * (1.0 / (1.0 + exp(-C))) * 1e6 * Qdtb / sum(Qdtb[1:end-1] * (t[2:end] - t[1:end-1]))
r = constant(py"np.load"(".npy files/r_.npy"))
z = constant(py"np.load"(".npy files/z_.npy"))

LoadError: UndefVarError: constant not defined

In [10]:
#θrz = Variable(fc_init([2,10,10,1]))
θrz = Variable(py"np.load"(".npy files/thetarz_.npy"))
f_r = fc([r z], [10, 10, 1], θrz)
f_r = 1.0 / (1.0 + exp(-f_r))
Qdff = f_r * tf.reshape(Qdot,(1,-1))

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\Michael\.julia\packages\PyCall\BcTLp\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'FileNotFoundError'>
FileNotFoundError(2, 'No such file or directory')
  File "C:\Users\Michael\.julia\conda\3\lib\site-packages\numpy\lib\npyio.py", line 384, in load
    fid = open(file, "rb")


In [ ]:
obs = py"np.load"(".npy files/obs.npy")
nt = size(timestep)[1]
inds = range(125, nt, step=20)
mTY = [m_; T; Y]
mTY = ADCME.constant(mTY)
loss = constant(0.0)
function condition(k, U, t, Qi)
    k <= nt - 1
end

function body(k, U, t, Qi)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    k1 = f(uk, Qi[k], t[k])
    k2 = f(uk + 0.5 * dt * k1, Qi[k], t[k] + 0.5 * dt)
    k3 = f(uk + 0.5 * dt * k2, Qi[k], t[k] + 0.5 * dt)
    k4 = f(uk + dt * k3, Qi[k], t[k] + dt) 
    u_new = uk + 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    U = write(U, k+1, u_new)
    k+1, U, t, Qi
end
k = constant(1, dtype=Int32)
t = constant(timestep)
Us= [TensorArray(nt) for i = 1:num_s]
i = 1
Us[i] = write(Us[i], 1, mTY)
_, U_out = while_loop(condition, body, [k, Us[i], t, Qdff[i,:]])
U_array = set_shape(stack(U_out), (nt, N+2))
# for i =1:num_s
#     print(i)
#     Us[i] = write(Us[i], 1, mTY)
#     _, U_out=while_loop(condition, body, [k, Us[i], t, Qdff[i,:]])
#     U_array = set_shape(stack(U_out), (nt, N+2))
#     loss += sum((U_array[inds,1]-obs[inds,i])^2)
#     print(i)
# end

In [ ]:
sess = Session()
init(sess)
out = run(sess, U_array);

In [ ]:
Tt_cant = py"np.load"(".npy files/temperature.npy")
plot(timestep, Tt_cant, label =false)
plot!(timestep, out[:,2], label =false)
xlabel!("Time")
ylabel!("Temperature")

In [ ]:
# Up to "End of Code"

In [ ]:
function sigmoid(X)
    sigma = 1 ./(1 .+ exp.(.-X))
    return sigma , X 
end
function relu(X)
    rel = max.(0,X)
    return rel , X
end

In [ ]:
function init_param(layer_dimensions)

    param = Dict()
    
    for l=2:length(layer_dimensions)

        param[string("W_" , string(l-1))] = rand(layer_dimensions[l] ,
				layer_dimensions[l-1])*0.1
        param[string("b_" , string(l-1))] = zeros(layer_dimensions[l] , 1)
    
    end

    return param

end

In [ ]:
function forward_linear(A , w , b)

    Z = w*A .+ b
    cache = (A , w , b)

    return Z,cache

end

In [ ]:
function calculate_activation_forward(A_pre , W , b , function_type)

    if (function_type == "sigmoid")

        Z , linear_step_cache = forward_linear(A_pre , W , b)
        A , activation_step_cache = sigmoid(Z)

    elseif (function_type == "relu")

        Z , linear_step_cache = forward_linear(A_pre , W , b)
        A , activation_step_cache = relu(Z)

    end

    cache = (linear_step_cache , activation_step_cache)
    return A , cache

end

In [ ]:
function model_forward_step(X , params)

    all_caches = []
    A = X
    L = length(params)/2

    for l=1:L-1
        A_pre = A
        A , cache = calculate_activation_forward(A_pre , params[string("W_" , string(Int(l)))] , params[string("b_" , string(Int(l)))] , "relu")
        push!(all_caches , cache)
    end 
    A_l , cache = calculate_activation_forward(A , params[string("W_" , string(Int(L)))] , params[string("b_" , string(Int(L)))] , "sigmoid")
    push!(all_caches , cache)


    return A_l , all_caches 

end

In [ ]:
function cost_function(AL , Y)
    
    cost = -mean(Y.*log.(AL) + (1 .- Y).*log.(1 .- AL))

    return cost 

end

In [ ]:
function backward_linear_step(dZ , cache)

    A_prev , W , b = cache

    m = size(A_prev)[2]
    
    dW = dZ * (A_prev')/m
    db = sum(dZ , dims = 2)/m
    dA_prev = (W')* dZ
    return dW , db , dA_prev 

end

In [ ]:
function backward_relu(dA , cache_activation)
    return dA.*(cache_activation.>0)
end 

function backward_sigmoid(dA , cache_activation)
    return dA.*(sigmoid(cache_activation)[1].*(1 .- sigmoid(cache_activation)[1]))
end

In [ ]:
function backward_activation_step(dA , cache , activation)

    linear_cache , cache_activation = cache
    if (activation == "relu")

        dZ = backward_relu(dA , cache_activation)
        dW , db , dA_prev = backward_linear_step(dZ , linear_cache)

    elseif (activation == "sigmoid")

        dZ = backward_sigmoid(dA , cache_activation)
        dW , db , dA_prev = backward_linear_step(dZ , linear_cache)

    end 

    return dW , db , dA_prev

end 

In [ ]:
function model_backwards_step(A_l , Y , caches)

    grads = Dict()

    L = length(caches)

    m = size(A_l)[2]

    Y = reshape(Y , size(A_l))
    dA_l = (-(Y./A_l) .+ ((1 .- Y)./( 1 .- A_l)))
    current_cache = caches[L]
    grads[string("dW_" , string(L))] , grads[string("db_" , string(L))] , grads[string("dA_" , string(L-1))] = backward_activation_step(dA_l , current_cache , "sigmoid")
    for l=reverse(0:L-2)
        current_cache = caches[l+1]
        grads[string("dW_" , string(l+1))] , grads[string("db_" , string(l+1))] , grads[string("dA_" , string(l))] = backward_activation_step(grads[string("dA_" , string(l+1))] , current_cache , "relu")

    end 

    return grads 

end

In [ ]:
function update_param(parameters , grads , learning_rate)

    L = Int(length(parameters)/2)

    for l=0:(L-1)

        parameters[string("W_" , string(l+1))] -= learning_rate.*grads[string("dW_" , string(l+1))]
        parameters[string("b_",string(l+1))] -= learning_rate.*grads[string("db_",string(l+1))]

    end 

    return parameters

end